In [1]:
import pandas as pd
import disamby.preprocessors as pre
from disamby import Disamby

In [97]:
#Dismbiguate KFUPM faculty members names with the names of the manually curated faculty data
data1 = pd.read_csv(r'/Users/nadee/Documents/1Graduate Studies/Term 2/NLP/Project/Code/KFUPM_Faculty_Departments.csv') 
data2 = pd.read_csv(r'/Users/nadee/Documents/1Graduate Studies/Term 2/NLP/Project/Code/authors_depts.csv') 

dataFrame1 = pd.DataFrame(data1, columns=['author', 'affiliation'])
dataFrame2 = pd.DataFrame(data2, columns=['author', 'affiliation'])

In [98]:
authors1 = dataFrame1["author"]
authors2 = dataFrame2["author"]

affiliation1 = dataFrame1["affiliation"]
affiliation2 = dataFrame2["affiliation"]

authors_list = authors1.tolist()
authors_list2 = authors2.tolist()

affiliations_list = affiliation1.tolist()
affiliations_list2 = affiliation2.tolist()

#Combine the two lists to find name disambiguation
authors_list.extend(authors_list2)

affiliations_list.extend(affiliations_list2)


for x in range(len(authors_list)):
    authors_list[x] = authors_list[x].strip().title()
    
print(len(authors_list))

1167


In [99]:
df = pd.DataFrame({
    'author': authors_list},
)

In [100]:
# define the pipeline to process the strings, note that the last step must return
# a tuple of strings
pipeline = [
    pre.normalize_whitespace,
    pre.remove_punctuation,
    lambda x: pre.trigram(x) + pre.split_words(x)
]

In [101]:
dis = Disamby(df, pipeline)

In [102]:
unique_authors = dis.disambiguated_sets(threshold=0.5)

100%|█████████████████████████████████████████████████████████████████████████████| 1167/1167 [00:06<00:00, 188.51it/s]


In [106]:
for x in unique_authors:
    if len(x)>1:
        print('Length:', len(x), "Unique:", x)
        print(x, type(x))
        s = list(x) 
        #print("s", s)
        for i in range(len(s)):
            if i == 0:
                #print(x, "Before", authors_list[s[i]])
                authors_list[s[i]] = authors_list[s[i]]
                #print(x, "After", authors_list[s[i]])
            else:
                #print(s[i], authors_list[s[i]])
                #print('Pre:', authors_list[s[i]], 'Post:', authors_list[s[i-1]] )
                print(x, "Before", authors_list[s[i]], affiliations_list[s[i]])
                #print('the matched:', affiliations_list[s[i-1]])
                if affiliations_list[s[i]] == affiliations_list[s[i-1]]:
                    authors_list[s[i]] = authors_list[s[i-1]]
                    print(x, "After", authors_list[s[i-1]], affiliations_list[s[i-1]])

Length: 2 Unique: {784, 38}
{784, 38} <class 'set'>
{784, 38} Before Mansour Mohammed Alhazmi Architectural Engineering Department
Length: 4 Unique: {49, 516, 470, 367}
{49, 516, 470, 367} <class 'set'>
{49, 516, 470, 367} Before Fahad Al-Gahtani Information and Computer Science Department
{49, 516, 470, 367} Before Ibrahim M. Algahtani Accounting and Finance Department
{49, 516, 470, 367} Before Al-Gahtani Muhammad Ali Electrical Engineering Department
Length: 3 Unique: {1113, 201, 1089}
{1113, 201, 1089} <class 'set'>
{1113, 201, 1089} Before Abdullah Faisal Abdullah Al-Ahmadi Chemistry Department
{1113, 201, 1089} After Abdullah Faisal Al-Ahmadi Chemistry Department
{1113, 201, 1089} Before Abdullah Faisal Al-Ahmadi Chemistry Department
{1113, 201, 1089} After Abdullah Faisal Al-Ahmadi Chemistry Department
Length: 2 Unique: {335, 55}
{335, 55} <class 'set'>
{335, 55} Before Hasan M. Al-Ahmadi Civil Engineering Department
Length: 2 Unique: {64, 967}
{64, 967} <class 'set'>
{64, 967} 

In [107]:
from pandas import DataFrame

df_authors = DataFrame(authors_list,columns=['author'])
df_affiliations = DataFrame(affiliations_list,columns=['affiliation'])

table = {"author": [], "affiliation": []}

for x in range(len(authors_list)):
    table["author"].append(df_authors['author'][x])
    table["affiliation"].append(df_affiliations['affiliation'][x])

In [108]:
len(authors_list)

1167

In [110]:
df=pd.DataFrame.from_dict(table,orient='index').transpose()

df.sort_values("author", inplace = True) 
  
# dropping ALL duplicte values 
df.drop_duplicates(subset ="author", keep = False, inplace = True)

df.to_csv('KFUPM_Faculty_Disambiguated.csv', index = False)